# TROPoe - Advanced Analysis

Welcome to the advanced notebook for the Tropospheric Remotely Observed Profiling via Optimal Estimation (TROPoe) agorithm. This notebook will go into more depth about some of the more important diagnostic variables we include in the file. Many times, if a retrieval looks _strange_ (and this can happen fairly often), then these diagnostic variables can give you a clue as to why. It is really important you know how the optimal estimation and TROPoe algorithm works to understand what is going on below. I highly reccomend reading Maahn et al. (2020) to get a sense of the assumptions that goes into optimal estimation. 

As a quick review, optimal estimation is a way to transform indirect observations of the atmosphere ($\mathbf{Y}$) into our typical atmospheric variables, or state vector ($\mathbf{X}$). To do this, we need some sort of forward model, $F(\mathbf{X})$, which converts the state vector into the same observation type as contained in $\mathbf{Y}$. We also need some sort of prior dataset to help the retrieval. 

References: 

- Maahn, M., D. D. Turner, U. Löhnert, D. J. Posselt, K. Ebell, G. G. Mace, and J. M. Comstock, 2020: Optimal Estimation Retrievals and Their Uncertainties: What Every Atmospheric Scientist Should Know. Bull. Amer. Meteor. Soc., 101, E1512–E1523, https://doi.org/10.1175/BAMS-D-19-0027.1.

- Turner, D. D., and U. Löhnert, 2014: Information Content and Uncertainties in Thermodynamic Profiles and Liquid Cloud Properties Retrieved from the Ground-Based Atmospheric Emitted Radiance Interferometer (AERI). J. Appl. Meteor. Climatol., 53, 752–771, https://doi.org/10.1175/JAMC-D-13-0126.1.

- Turner, D. D., and W. G. Blumberg, 2019: Improvements to the AERIoe Thermodynamic Profile Retrieval Algorithm. IEEE J. Sel. Top. Appl. Earth Observations Remote Sensing, 12, 1339–1354, https://doi.org/10.1109/JSTARS.2018.2874968.

### Contents
- CBH Plotting (explain default cbh (sensitivity to low LWP))
- Go over the different RMS values
- Observation vector and the flags
- Prior/posterior covariance 
- Degrees of freedom/effective resolution/Shannon information content
- dindices
- Averaging kernel

In [70]:
%matplotlib widget

from datetime import datetime

import cmocean
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np

from utils import timeheight, corr_plot, cov2corr

from ipywidgets import interact


### Advanced variables

Lets take another look at the file we plotted in the basic notebook. Again, we will use opendap to 

In [71]:
nc = Dataset('https://data.nssl.noaa.gov/thredds/dodsC/FRDD/CLAMPS/clamps/clamps1/processed/clampstropoe10.aeri_mwr.v2.C1/clampstropoe10.aeri_mwr.v2.C1.20190920.001005.cdf')

In [41]:
for v in nc.variables:
    print(f"**{v}**: {nc.variables[v].long_name} -- {nc.variables[v].dimensions}\n")

**base_time**: Epoch time -- ()

**time_offset**: Time offset from base_time -- ('time',)

**hour**: Time -- ('time',)

**qc_flag**: Manual QC flag -- ('time',)

**height**: height -- ('height',)

**temperature**: temperature -- ('time', 'height')

**waterVapor**: water vapor mixing ratio -- ('time', 'height')

**lwp**: liquid water path -- ('time',)

**lReff**: liquid water effective radius -- ('time',)

**iTau**: ice cloud optical depth (geometric limit) -- ('time',)

**iReff**: ice effective radius -- ('time',)

**co2**: carbon dioxide concentration -- ('time', 'gas_dim')

**ch4**: methane concentration -- ('time', 'gas_dim')

**n2o**: nitrous oxide concentration -- ('time', 'gas_dim')

**sigma_temperature**: 1-sigma uncertainty in temperature -- ('time', 'height')

**sigma_waterVapor**: 1-sigma uncertainty in water vapor mixing vapor -- ('time', 'height')

**sigma_lwp**: 1-sigma uncertainty in liquid water path -- ('time',)

**sigma_lReff**: 1-sigma uncertainty in liquid water effe

Before, I filtered out a lot of the variables that were not directly related to the thermodynamic profile. Now we need to be familiar with the whole list. A few things to note:

- TROPoe retrieves cloud properties like liquid water path, effective radius of liquid water and ice
- TROPoe also has the ability to retrieve trace gas profiles (CO2, CH4, N2O). This is experiemental and should not be used at the moment
- The retrieval contains cloud based height information
- The output files contain the full observation vector as well as the forward calculation of the derived solution (i.e. $F(\mathbf{X}_{op})$)
- The output files contain the prior and posterior covariance matrices 

Here is a list of all the variables and their description from the netcdf file:

**base_time**: Epoch time -- ()

**time_offset**: Time offset from base_time -- ('time',)

**hour**: Time -- ('time',)

**qc_flag**: Manual QC flag -- ('time',)

**height**: height -- ('height',)

**temperature**: temperature -- ('time', 'height')

**waterVapor**: water vapor mixing ratio -- ('time', 'height')

**lwp**: liquid water path -- ('time',)

**lReff**: liquid water effective radius -- ('time',)

**iTau**: ice cloud optical depth (geometric limit) -- ('time',)

**iReff**: ice effective radius -- ('time',)

**co2**: carbon dioxide concentration -- ('time', 'gas_dim')

**ch4**: methane concentration -- ('time', 'gas_dim')

**n2o**: nitrous oxide concentration -- ('time', 'gas_dim')

**sigma_temperature**: 1-sigma uncertainty in temperature -- ('time', 'height')

**sigma_waterVapor**: 1-sigma uncertainty in water vapor mixing vapor -- ('time', 'height')

**sigma_lwp**: 1-sigma uncertainty in liquid water path -- ('time',)

**sigma_lReff**: 1-sigma uncertainty in liquid water effective radius -- ('time',)

**sigma_iTau**: 1-sigma uncertainty in ice cloud optical depth (geometric limit) -- ('time',)

**sigma_iReff**: 1-sigma uncertainty in ice effective radius -- ('time',)

**sigma_co2**: 1-sigma uncertainty in carbon dioxide concentration -- ('time', 'gas_dim')

**sigma_ch4**: 1-sigma uncertainty in methane concentration -- ('time', 'gas_dim')

**sigma_n2o**: 1-sigma uncertaintiy in nitrous oxide concentration -- ('time', 'gas_dim')

**converged_flag**: convergence flag -- ('time',)

**gamma**: gamma parameter -- ('time',)

**n_iter**: number of iterations performed -- ('time',)

**rmsr**: root mean square error between AERI obs in the observation vector and the forward calculation -- ('time',)

**rmsa**: root mean square error between observation vector and the forward calculation -- ('time',)

**rmsp**: root mean square error between prior T/q profile and the retrieved T/q profile -- ('time',)

**chi2**: Chi-square statistic of Y vs. F(Xn) -- ('time',)

**convergence_criteria**: convergence criteria di^2 -- ('time',)

**dfs**: degrees of freedom of signal -- ('time', 'dfs')

**sic**: Shannon information content -- ('time',)

**vres_temperature**: Vertical resolution of the temperature profile -- ('time', 'height')

**vres_waterVapor**: Vertical resolution of the water vapor profile -- ('time', 'height')

**hatchOpen**: Flag indicating if the AERIs hatch was open -- ('time',)

**cbh**: Cloud base height -- ('time',)

**cbh_flag**: Flag indicating the source of the cbh -- ('time',)

**pressure**: derived pressure -- ('time', 'height')

**theta**: potential temperature -- ('time', 'height')

**thetae**: euivalent potential temperature -- ('time', 'height')

**rh**: relative humidity -- ('time', 'height')

**dewpt**: dew point temperature -- ('time', 'height')

**dindices**: derived indices -- ('time', 'index_dim')

**sigma_dindices**: 1-sigma uncertainties in the derived indices -- ('time', 'index_dim')

**obs_flag**: Flag indicating type of observation for each vector element -- ('obs_dim',)

**obs_dim**: Dimension of the observation vector -- ('obs_dim',)

**obs_vector**: Observation vector Y -- ('time', 'obs_dim')

**obs_vector_uncertainty**: 1-sigma uncertainty in the observation vector (sigY) -- ('time', 'obs_dim')

**forward_calc**: Forward calculation from state vector (i.e., F(Xn)) -- ('time', 'obs_dim')

**arb**: arbitrary dimension -- ('arb',)

**Xop**: optimal solution -- ('time', 'arb')

**Sop**: covariance matrix of the solution -- ('time', 'arb', 'arb')

**Akernal**: averaging kernal -- ('time', 'arb', 'arb')

**Xa**: prior mean state -- ('arb',)

**Sa**: prior covariance -- ('arb', 'arb')

**lat**: latitude -- ()

**lon**: longitude -- ()

**alt**: altitude -- ()

### Cloud base height
Now lets revisit the time height plot from the TROPoe Basics notebook. This time, we'll plot the cloud base height (CBH) on top of the color fill. When using TROPoe data from CLAMPS, the CBH is calculated from the Doppler lidar backscatter during vertical stares. If no cloud is detected throughout the profile, or if there is no lidar data available, we set a default CBH. This is typically set to 1 or 2 km. The reason we include this even though there is no cloud detected by the lidar is due to the high sensitivity of the AERI to low values of LWP (<60 g~m${-2}$). Clouds with low LWP are often 'invisible' to lidar and celiometers thus we assume one to be present. If the sky is actually cloud free, then the retrieved LWP still should go to zero (within the proper uncertainty levels).

Looking at the figure below, CBH is relatively low throughout the period. Above the CBH point we see a lot of streaking in the retrieval. This is actually expected and important to know. Generally, we only trust AERI retrievals below cloud base height since the AERI can not 'see' into clouds. MWRs are able to 'see' a into some clouds, but we still typically trust the data below CBH more. 

We can also see what happened during the 10Z and 15Z time frame based on the constant CBH at 2 km. During this time period, the lidar was not operational and the cloud base height defaulted to 2 km. Extrapolating the data from before the data outage to the data after the outage, we can conclude that 2 km is much higher than the actual CBH during the outage, thus the retrieval looks drastically different during this period. 

In [42]:
# Get the times and make sure they are sorted
t = [datetime.utcfromtimestamp(d) for d in (nc['base_time'][:]+nc['time_offset'][:])]
h = nc['height'][:]

# Create the figure 
fig, (temp_ax, wvmr_ax) =  plt.subplots(2, sharex=True)
fig.set_figheight(7.5)
fig.set_figwidth(12.5)

X, Y = np.meshgrid(t, h)

timeheight(X, Y, nc['temperature'][:].T, 't', temp_ax, zmin=0, zmax=3, datamin=0, datamax=30)
timeheight(X, Y, nc['waterVapor'][:].T, 'q', wvmr_ax, zmin=0, zmax=3, datamin=0, datamax=15)

ind = np.where(nc['cbh'] == 0)

temp_ax.scatter(t, nc['cbh'][:], color='k', marker='x')
wvmr_ax.scatter(t, nc['cbh'][:], color='k', marker='x')

<ipython-input-42-229d530f6d3e>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (temp_ax, wvmr_ax) =  plt.subplots(2, sharex=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/tyler.bell/Projects/CLAMPS_Demos/utils.py:89: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  c = ax.pcolormesh(time, height, data, vmin=datamin, vmax=datamax, cmap=cm, **kwargs)


### RMS Error

Another way to evaluate how well the retrieval is performing is to look at the root mean square (RMS) error between the actual observations and the synthetic observations calculated by the forward model. These values are included in the netcdf file. You will notice that there are three different RMS varibles: rmsr, rmsa, and rmsp. These are all slightly different in meaning. It is particularly difficult to interpret RMS from these retrievals since the observation vector often has measurements from many different instruments and they all are not necessicairly the same unit. 

- **RMSR**: RMS of only the AERI observations included in the observation vector. 
- **RMSA**: RMS of the full observation vector. So this can include mixed units and changes drastically based on what observations are included in the retrieval. It is often difficult to interpret physically due to the mixed units. 
- **RMSP**: RMS of your retrieval compared to the prior. So this can give you information about how far away from the climatology a solution may be. 

An example of each is shown below. 


In [43]:
plt.figure(figsize=(10, 5))
plt.plot(t, nc['rmsr'][:], label="RMSR")
plt.plot(t, nc['rmsa'][:], label="RMSA")
plt.plot(t, nc['rmsp'][:], label="RMSP")
plt.ylabel("RMS")
plt.xlabel("Time UTC")
plt.title(t[0])
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### The forward model and observation vector

We can also look at these RMS values in context of the observation vector. The below figure adds the difference between the observation vector $\mathbf{Y}$ and the forward model $F(\mathbf{X})$. Ideally, all the values are zero. However, there are often differences. Remember, this is again mixed units, so be sure to look at the `obs_flag` documentation to know what you are looking at. If we look below, you will see the following observations are included in the retrieval:

- AERI 
- MWR
- Surface met T
- Surface met WVMR
- RAP T above 4km 
- RAP WVMR above 4 km

In [61]:
nc['obs_flag']

<class 'netCDF4._netCDF4.Variable'>
int16 obs_flag(obs_dim)
    long_name: Flag indicating type of observation for each vector element
    units: mixed units -- see comments below
    value_01: cm^(-1) (i.e., wavenumber)
    value_02: Brightness temperature in K from a zenith-microwave radiometer
    value_05: Surface met temeprature in C from Microwave radiometer met station
    value_05_comment1: Surface met station is 3 m above height=0 level
    value_05_comment2: Adding 5.0 C to uncertainty to account for representativeness error
    value_06: Surface met water vapor in g/kg from Microwave radiometer met station
    value_06_comment1: Surface met station is 3 m above height=0 level
    value_07: Temperature in C from NWP model output from RUC/RAP (NCEI THREDDS) at 45.6268162051015 degN, -91.9173866962401 degE
    value_08: Water vapor in g/kg from NWP model output from RUC/RAP (NCEI THREDDS) at 45.6268162051015 degN, -91.9173866962401 degE
unlimited dimensions: 
current shape = (3

In [63]:
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.set_figheight(7.5)
fig.set_figwidth(10)

ind = 25
@interact(ind=(0, nc.dimensions['time'].size-1))
def plot_fwd(ind):
    ax1.cla()
    ax1.plot(t, nc['rmsr'][:], label="RMSR")
    ax1.plot(t, nc['rmsa'][:], label="RMSA")
    ax1.plot(t, nc['rmsp'][:], label="RMSP")
    
    ax1.vlines(t[ind], 0, 100, color='k')
    ax1.set_ylim(0, 10)
    ax1.legend()
    
    ax2.cla()
    
    for dim in np.unique(nc['obs_flag'][:]):
        foo = np.where(nc['obs_flag'][:] == dim)[0]
        ax2.scatter(np.arange(0, nc.dimensions['obs_dim'].size)[foo], nc['forward_calc'][:][ind, foo]-nc['obs_vector'][:][ind, foo], marker='.', cmap='Accent', label=dim)
    
    ax2.set_xlabel("Observation index")
    ax2.set_ylabel("F(X) - Y")
    ax2.legend()

    plt.grid()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=71, description='ind', max=142), Output()), _dom_classes=('widget-intera…

### Effective Resolution    

One of the interesting things we get out of a retrieval like TROPoe is all of the intermediate matricies in the calculation. They contain a wealth of information about how well the retrieval is performing.  An important point of this retrieval is that although we have specified 55 independent levels for both temperature and water vapor mixing ratio, there are not 55 independent pieces of information. One of the things we can calculate from an optimal estimation retrieval is the effective resolution. These are directly included in output files. 

If we look at the effective resolution from the file we have been working with, we see how quickly AERI and MWR observations lose information content. The main contributers to the information content in the profile below 4 km are the observations from the AERI and MWR. Note that often, toward the top of the average boundary layer, the effective resolution is approximately the depth of the boundary layer! This highlights the importantance of a quality prior dataset. We rely on that prior and its covariances to effectively retrieve realistic profiles. Around 4 km, the data from the RAP kicks in and we see a large decrease in effective resolution. 

In [66]:

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figheight(5)
fig.set_figwidth(10)


@interact(ind=(0, nc.dimensions['time'].size-1))
def doplot_vres(ind):
    t_op = nc['temperature'][:][ind]
    t_err = nc['sigma_temperature'][:][ind]
    ax1.cla()
    ax1.plot(nc['vres_temperature'][:][ind]*1e3, nc['height'][:]*1e3, '-o', color='maroon')
#     ax1.fill_betweenx(nc['height'][:], t_op+t_err, t_op-t_err, color='maroon', alpha=.2)
#     ax1.set_ylim(0, 3000)
#     ax1.set_xlim(0, 5000)
    ax1.grid()
    ax1.set_xlabel("Effective Resolution [m]")
    ax1.set_ylabel("Height [m]")

    
    w_op = nc['waterVapor'][:][ind]
    w_err = nc['sigma_waterVapor'][:][ind]
    ax2.cla()
    ax2.plot(nc['vres_waterVapor'][:][ind]*1e3, nc['height'][:]*1e3, '-o', color='C0')
    ax2.fill_betweenx(nc['height'][:], w_op+w_err, w_op-w_err, color='C0', alpha=.2)
#     ax2.set_ylim(0, 3000)
#     ax2.set_xlim(0, 5000)  
    ax2.grid()
    ax2.set_xlabel("Effective Resolution [m]")
#     ax2.set_ylabel("Height [m]")
#     plt.tight_layout()
    plt.suptitle(t[ind])
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=71, description='ind', max=142), Output()), _dom_classes=('widget-intera…

### Correlation Matrices  

One final way we can evaluate the how well a retrieval is performing is by peeking at the posterior correlation matrices that are produced. It is informative to first look at what the prior correlation matrix looks like. I've included a function in `utils.py` that plots the prior/posterior _covariance_ matrix that is included in the files as temperature and moisture _correlation_ matrices. Directly below is what the correlation calculated from the prior looks like:

In [69]:
corr_plot(cov2corr(nc['Sa'][:]), nc['height'][:])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.0000001


Now ideally, after the retrieval is performed, we would have a diagonal matrix with all off diagonal components being zero. That would mean we have perfect information content at every level. However, as we saw above, we don't get enough information from our instruments for this to happen. We can plot the posterier correlation matricies with the same `cov2corr` function. Note that these are relatively diagonal compared to the prior correlation plots and have much less level-to-level correlation. However, like we saw in the effective resolution plots, we have a lot more off-diagonal correlation typically below 4 km due to the AERI/MWR losing information content. 

In [59]:
@interact(ind=(0, nc.dimensions['time'].size-1))
def do_corr_plot(ind):
    plt.close()
    corr_plot(cov2corr(nc['Sop'][:][ind]), nc['height'][:])

interactive(children=(IntSlider(value=71, description='ind', max=142), Output()), _dom_classes=('widget-intera…